# Nearest Neighbour `VS` Genetic Algorithm

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
from matplotlib.widgets import Cursor
%matplotlib qt

In [2]:
# df = pd.read_excel("15-Points.xlsx")
# df['test']= np.array(range(1,16))
# df

In [19]:
def input_points ():
    points = []
    
    # display a plot and wait for the user to click on num_points points
    fig, ax = plt.subplots()
    plt.title('Input Points')
    plt.xlim(0, 10)
    plt.ylim(0, 10)
    plt.show()

    cursor = Cursor(ax, horizOn=True, vertOn=True, useblit=True, color='black', linewidth=1)

    # take input fro
    while True:
        # get the first tuple (first point)
        point = plt.ginput(1)
        if point:
            point=point[0]
        else:
            break

        points.append(point)
        plt.plot(point[0], point[1], 'bo')
        plt.draw()
        
    plt.close()
    
    return points

In [4]:
def plot_tsp(data, Best_solution,Best_cost,title):
    fig, ax = plt.subplots()
    x = data['x']
    y = data['y']
    plt.scatter(x, y, color = 'b')
    plt.title(title)
    x_init, y_init = data[data['City']==Best_solution[0]]['x'], data[data['City']==Best_solution[0]]['y']
    plt.scatter(x_init, y_init, color = 'red')
    plt.text(x_init - 0.5 , y_init + 0.6 , "Start" , color = 'm')
    

    x = []
    y = []
    x.append(x_init)
    y.append(y_init)
    TSP_plot = plt.plot(x, y, color = 'r')

    for i, city in enumerate(Best_solution[0:]):
        city_x, city_y = data[data['City']==Best_solution[i]]['x'], data[data['City']==Best_solution[i]]['y']
        x.append(city_x)
        y.append(city_y)  
        plt.setp(TSP_plot, xdata = x, ydata = y)
        plt.pause(0.3)

    x.append(x_init)
    y.append(y_init)
    plt.setp(TSP_plot, xdata = x, ydata = y)
    plt.xlabel(f"Best_cost: {round(Best_cost,5)}" ,color = 'm')
    plt.pause(0.3)
#     plt.text(0.5 , 0.5 , f"Best_cost: {round(Best_cost,5)}" , color = 'm')
#     plt.xlabel(f"Best_cost: {round(Best_cost,5)}" ,color = 'm')
    
    

In [5]:
def euclidean_distance(df,c1,c2):
    x1 = df.loc[df['City'] == c1, 'x'].iloc[0]
    y1 = df.loc[df['City'] == c1, 'y'].iloc[0]
    x2 = df.loc[df['City'] == c2, 'x'].iloc[0]
    y2 = df.loc[df['City'] == c2, 'y'].iloc[0]
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

In [6]:
def distance(df):
    count = len(df)
    dist_arr = np.zeros((count,count))
    for c1 in range(0,count-1):
        for c2 in range(c1+1,count):
            dist_arr[c1,c2]= euclidean_distance(df,c1,c2)
            dist_arr[c2,c1] =dist_arr[c1,c2]

        dist_arr[c1,c1] = np.inf
    dist_arr[c1+1,c1+1] = np.inf

    return dist_arr

In [7]:
#########################  Genetic Algorithm ##################################

In [8]:
class Chromosome :
    def __init__(self,chrom):
        self.chrom = chrom
        self.distList = []
        self.cost = None

    def evaluateFitness(self,dist_arr) :
        self.distList = []
        for i in range(0,len(self.chrom)-1) :
            dist = dist_arr[self.chrom[i]][self.chrom[i+1]]
            self.distList.append(dist)
        self.cost = sum(self.distList)




In [9]:
#uniform Crossover
def Crossover(crossover_list,count,dist_arr):
    uniform = random.choices([0,1], k=count)
    newgen = [None]*len(crossover_list)
    for c in range(0,len(crossover_list),2):
        newgen[c]=Chromosome(crossover_list[c].chrom)
        newgen[c+1]=Chromosome(crossover_list[c].chrom)
        for i in range(len(uniform)) :
            if uniform[i] == 1:
                newgen[c].chrom[i],newgen[c+1].chrom[i] = newgen[c+1].chrom[i] , newgen[c].chrom[i]
        newgen[c].evaluateFitness(dist_arr)
        newgen[c+1].evaluateFitness(dist_arr)

    return newgen



# Mutation

def Swapping(chrom):
    sw_ndx = random.sample(range(len(chrom)), k=2)
    newChrom= chrom.copy()
    newChrom[sw_ndx[0]],newChrom[sw_ndx[1]] = newChrom[sw_ndx[1]] ,newChrom[sw_ndx[0]]
    return newChrom

def Scramble(chrom):
    left = random.randint(0, len(chrom)-2)
    right = random.randint(left+1, len(chrom))
    newChrom= chrom.copy()
    new = random.sample(newChrom[left : right], len(newChrom[left : right]))
    newChrom = np.array(newChrom)
    newChrom[left : right] = new
    newChrom= newChrom.tolist()
    return newChrom

def Inversion(chrom):
    left = random.randint(0, len(chrom)-2)
    right = random.randint(left+1, len(chrom))
    newChrom= chrom.copy()
    new = newChrom[left : right]
    new.reverse()
    newChrom = np.array(newChrom)
    newChrom[left : right] = new
    newChrom= newChrom.tolist()
    return newChrom

def Mutation (M_list,dist_arr):
    newgen = [None]*len(M_list)
    for i in range(len(M_list)) :
        newgen[i] = Chromosome(M_list[i].chrom.copy())
        r = random.randint(0, 3)
        if r == 0:
            newgen[i].chrom = Swapping(newgen[i].chrom).copy()
        elif r == 1:
            newgen[i].chrom = Scramble(newgen[i].chrom).copy()
        else:
            newgen[i].chrom = Inversion(newgen[i].chrom).copy()

        newgen[i].evaluateFitness(dist_arr)

    return newgen



In [10]:
Inversion([3,5,6,7,8,4])

[3, 4, 8, 7, 6, 5]

In [11]:
def Genetic(data,pop_size, gen_count, elit, cross_num , mut_num):
    
    #distance
    dist_arr = distance(data)
    count = len(data)
    
    #Initial Population 
    city_n=[i for i in range(0,count)]
    pop_list = [None]*pop_size
    for p in range(pop_size): #Generate Chromosome 
        chrom = random.sample(city_n, k=len(data))
        pop_list[p] = Chromosome(chrom)
        pop_list[p].evaluateFitness(dist_arr)
    #sort
    pop_list = sorted(pop_list, key=lambda d: d.cost)
    
    # Initialization Generations
    for i in range(gen_count):
        next_pop = []

        #select elitism
        elit_list = pop_list[0:elit]

        # Perform crossover
        select_crossover = random.sample(pop_list, k=cross_num)
        crossover_list = Crossover(select_crossover,count,dist_arr).copy()

        # Perform mutation
        select_Mutation = random.sample(crossover_list.copy(), k=mut_num)
        Mutation_list = Mutation(select_Mutation,dist_arr).copy()

        # select Population size
        next_pop =  elit_list + pop_list + crossover_list + Mutation_list
        next_pop = sorted(next_pop, key=lambda d: d.cost )

        pop_list = next_pop[0:pop_size]

    # Best solution
    print("Best Path : ",pop_list[0].chrom)
    print("Path dist : ", np.round(pop_list[0].distList , 3).tolist())
    print("Best cost : ",pop_list[0].cost)
    
    Best_Path = pop_list[0].chrom.copy()
    Path_dist = np.round(pop_list[0].distList.copy() , 3).tolist()
    Best_cost = pop_list[0].cost
    
    plot_tsp(data, Best_Path,Best_cost,'Genetic algorithm')
    
    
    return Best_Path, Path_dist,Best_cost

In [12]:
#########################  Nearest Neighbour  ##################################

In [13]:
def Nearest(data,start):
    
#     points = input_points()
#     data = pd.DataFrame(points, columns = ['x', 'y'])
#     data['City'] = np.random.permutation(len(points))
    
    dist_arr = distance(data)
    count = len(data)
    
    Best_cost=0
    Best_Path=[start]
    Path_dist=[]
    
    for c in range(count-1):
        city =Best_Path[-1]
        dist = dist_arr[city].min()
        Best_cost+=dist
        new_city = dist_arr[city].argmin()
        dist_arr[city,:]= np.inf
        dist_arr[:,city]= np.inf

        Best_Path.append(new_city)
        Path_dist.append(dist)



    print("Best Path : ",Best_Path)
    print("Path dist : ", Path_dist)
    print("Best cost : ",Best_cost)
    
    plot_tsp(data, Best_Path,Best_cost,'Nearest Neighbour')
    
    return Best_Path, Path_dist , Best_cost
    

In [54]:
######################### call fun ######################

In [55]:
points = input_points()
data = pd.DataFrame(points, columns = ['x', 'y'])
data['City'] = np.random.permutation(len(points))

In [56]:
#Nearest Neighbour parameters
start =0
Best_Path, Path_dist,Best_cost= Nearest(data,start)

Best Path :  [0, 12, 2, 8, 16, 9, 14, 13, 3, 4, 6, 1, 11, 7, 15, 10, 5]
Path dist :  [2.011788107965596, 2.3118273918943926, 1.9153225806451615, 1.8471680907267507, 1.9365067672931584, 2.028250830917722, 1.7927446849118747, 3.418458286174299, 2.5520866098217696, 2.1734428252979154, 1.475913758042178, 0.0, 1.7455460217371468, 2.224469788019616, 1.1156093286121191, 4.584022018072745]
Best cost :  33.13315709013245


In [57]:
# print()

In [58]:
#Genetic algorithm parameters
pop_size= 100 
gen_count=200 
elit = 3
cross_num = 30
mut_num = 8

Best_Path, Path_dist,Best_cost= Genetic(data,pop_size, gen_count, elit, cross_num , mut_num)

Best Path :  [9, 16, 8, 2, 0, 12, 10, 15, 7, 11, 1, 6, 5, 4, 3, 14, 13]
Path dist :  [1.937, 1.847, 1.915, 2.449, 2.012, 3.131, 1.116, 2.224, 1.746, 0.0, 1.476, 1.746, 2.517, 2.552, 1.921, 1.793]
Best cost :  30.380704534297163
